In [1]:
import os
from bs4 import BeautifulSoup
import email
import re
import string
from elasticsearch import Elasticsearch 
# from nltk.corpus import stopwords

In [8]:
es=Elasticsearch()

In [3]:
#path of the data file
path=('C:/Users/Nikhar/Downloads/Assignment/InformationRetrievalCS6200/HW7/full/trec07p/data/')

In [2]:
#Reading the file with mapping for each file with respect to being spam or ham
spam_dict={}
index_path='C:/Users/Nikhar/Downloads/Assignment/InformationRetrievalCS6200/HW7/full/trec07p/full/'
for line in os.listdir(index_path):
    file=open(index_path+line,"r", encoding="ISO-8859-1")
    for line in file:
        line=line.strip().split(' ')
        fname=line[1].split('/')
        spam_dict[fname[2]]=line[0]
       

In [6]:
#naming the index for elastic search
index='trec_spam'

In [20]:
#creating the index with elastic search
es.indices.create(index=index,
                      body={
                          'settings': {
                              'index': {
                                  'store': {
                                      'type': "fs"
                                  },
                                  'number_of_shards': 1,
                                  'number_of_replicas': 1
                              },
                              'analysis': {
                                 "analyzer": {
                                   "fulltext_analyzer": {
                                       "type": "custom",
                                       "tokenizer": "whitespace",
                                          "filter": [
                                               "lowercase",
                                                    ]
                                                  }
                                            }
                              }
                          },
                          "mappings": {
                              "document": {
                                  "properties": {
                                      "file_name": {
                                          "type": "text",
                                          "store": True,
                                          "index": True
                                      },
                                      "body": {
                                          "type": "text",
                                          "store": True,
                                          "index": True,
                                          "fielddata":True,
                                          "term_vector": "with_positions_offsets_payloads",
                                          "analyzer": "fulltext_analyzer"
                                      },
                                      "label":{
                                          "type": "keyword",
                                          "store": True,
                                          "index": True
                                      }
                                      }
                                  }
                              }
                          }
         )

{'acknowledged': True, 'shards_acknowledged': True}

In [15]:
#reading the stopwords from stoplist file
with open("C:/Users/Nikhar/Downloads/Side_work/spam/stoplist.txt") as sfile:
    stopWords = sfile.readlines()
stopWords = set(map(str.strip, stopWords))
sfile.close()

In [21]:
caught=0
count=0
path=('C:/Users/Nikhar/Downloads/Assignment/InformationRetrievalCS6200/HW7/full/trec07p/data/')
for filename in os.listdir(path):
#     print(filename)
    count+=1
    mail_type=spam_dict[filename]

    file=open(path+filename,"r", encoding="ISO-8859-1")
    mail_text=""
    for line in file:
        mail_text=mail_text+line
        mail_text= mail_text.replace('\n',' ').replace('\t',' ')

    
    mail_text=re.sub(' +',' ',mail_text)
    mail_text=mail_text.lower()
    soup=BeautifulSoup(mail_text,'lxml')
    text=soup.get_text()
    for p in string.punctuation:
        if p!='_'  and p!='\'' and p!='@' :
            text=text.replace(p,' ')
    text=text.replace(' com','.com')
    
    
    string_x=list()
    mail_text=text.split()
    for unigram in mail_text:
        if unigram in stopWords:
            string_x.append('')
        else:
            string_x.append(unigram)
    mail_text=' '.join(string_x)
    
    mail_text = re.sub('[0-9]+', '', mail_text)
    mail_text=mail_text.replace('   ',' ')
    mail_text=mail_text.replace('  ',' ')
    mail_text=mail_text.strip()
    
    if mail_type=="spam":
        label='yes'
        caught=caught+1
    else:
        label='no'
    
    
    doc={
        "file_name": filename,
        "body":mail_text,
        "label":label
    }
    
    res=es.index(index=index, doc_type='document', id=filename, body=doc,ignore=[404,400])

print('Total files indexed:'+str(count))
print('Total spam files indexed:'+str(caught))


Total files indexed:75419
Total spam files indexed:50199
